<a href="https://colab.research.google.com/github/nwang2208/Bioinformatics-Class-Projects/blob/homework-for-algorithms/HW7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

10.5 Q8

In [ ]:
Paga = 0.099
Pbga = 0.901
Pagb = 0.743
Pbgb = 0.257

text = 'ABBBAAABAABBBBAAAABBAABBABBBBBABAABBBBABAABAABBABA'

In [ ]:
Ppi = .5
for i in range(1, len(text)):
    if(text[i-1] == 'A'):
        if(text[i] == 'A'):
            Ppi = Ppi * Paga
        else:
            Ppi = Ppi * Pbga
    elif(text[i-1] == 'B'):
        if(text[i] == 'A'):
            Ppi = Ppi * Pagb
        else:
            Ppi = Ppi * Pbgb


In [ ]:
Ppi

10.5 Q10

In [ ]:
x = 'xzyyzxzzxxxyyxyyzxyzzzyyyzzyyxxyzyxzxxyxzxzzyzxyzy'
path = 'BBABBBBBAABAABBAAABABBBBAAABABABBABAABAAAABABBABAA'

Pxga = 0.303
Pyga = 0.296
Pzga = 0.401

Pxgb = 0.018
Pygb = 0.252
Pzgb = 0.73



In [ ]:
Ppgx = 1
for i in range(0, len(path)):
    if(path[i] == 'A'):
        if(x[i] == 'x'):
            Ppgx = Ppgx * Pxga
        elif(x[i] == 'y'):
            Ppgx = Ppgx * Pyga
        else:
            Ppgx = Ppgx * Pzga
    elif(path[i] == 'B'):
        if(x[i] == 'x'):
            Ppgx = Ppgx * Pxgb
        elif(x[i] == 'y'):
            Ppgx = Ppgx * Pygb
        else:
            Ppgx = Ppgx * Pzgb


In [ ]:
Ppgx

1.223882013580701e-31

10.6 Q7

In [ ]:
import numpy as np

In [ ]:
def viterbi(emission, transition, x):
    n = len(x) #length of x

    s = np.ones(shape = (2, n)) * -float('inf')
    pointers = [[False for e in range(n)] for s in range(2)]

    for state in range(2): #source
        s[state][0] =  np.log(1/2) + emission[state][x[0]]
        pointers[state][0] = -1

    for i in range(1, n):
        for state in range(2):
            for prev in range(2):
                p_total = emission[state,emission[i]] + transition[prev,state] + s[prev,i-1]
                if(p_total > s[state,i]):
                    s[state][i] = p_total
                    pointers[state,i] = prev

    score = -float('inf')
    for state in range(2):
        if s[state][n-1] > score:
            last = state
            score = s[state][n-1]
    path = [last]

    i = n-1
    while i>0:
        next = pointers[last][i]
        path.append(next)
        last = next
        i -= 1

    return path



In [ ]:
def parse_HMM(lines):
    emission = lines[0].strip()
    alphabet = lines[2].strip().split()
    emission = [int(alphabet.index(em)) for em in emission]
    states = lines[4].strip().split()
    S = len(states)
    T = np.array([line.split()[1:] for line in lines[7:7+S]], float)
    T = np.log(T)
    E = np.array([line.split()[1:] for line in lines[9+S:]], float)
    E = np.log(E)
    return(emission, alphabet, states, T, E)

In [ ]:
def viterbi_algorithm(emission, T, E, states, alphabet):
    """returns max likelihood hidden path for emission string, given HMM"""

    S = len(states)
    n = len(emission)
    # initialize Viterbi graph and backpointers matrices
    viterbi = np.ones(shape = (S, n)) * -float('inf')
    pointers = [[False for e in range(n)] for s in range(S)]

    # initialize first column of viterbi with: weight(node) = (Pr_emission)*(1/States)
    for state in range(S):
        viterbi[state][0] = np.log(1/S) + E[state][emission[0]]
        pointers[state][0] = -1

    # Fill viterbi graph using dynamic programming
    for i in range(1,n):
        for state in range(S):
            for prev in range(S):
                p_total = E[state][emission[i]] + T[prev][state] + viterbi[prev][i-1]
                # find max-weight path to current node
                if p_total > viterbi[state][i]:
                    viterbi[state][i] = p_total
                    pointers[state][i] = prev

    # start backtrack from max-likelihood state in last column of viterbi graph
    score = -float('inf')
    for state in range(S):
        if viterbi[state][n-1] > score:
            last = state
            score = viterbi[state][n-1]
    path = [last]

    # backtrack to recreate max likelihood hidden_path in reverse
    i = n-1
    while i > 0:
        next = pointers[last][i]
        path.append(next)
        last = next
        i -= 1
    # reverse string to get hidden_path solution to decoding problem
    return ''.join(str(states[state]) for state in path[::-1])

In [ ]:
with open("dataset_925972_7.txt") as f:
    lines = [line.strip() for line in f]
emission, alphabet, states, T, E = parse_HMM(lines)
#decoded_pi = viterbi(emission, T, E)
decoded_pi = viterbi_algorithm(emission, T, E, states, alphabet)

In [ ]:
decoded_pi

'CCBBBBBBBBBBCAAAAACACCAACCACAAAABBCAAAAACCCACBBBBBCCAACCBBBBCCCCAACCACCCCBBBCCAAACCCCCAAACCCCBBBBCAC'

10.7 Q4

In [ ]:
def outcome_likelihood(emission, T, E, states): # outcome-likelihood of HMM emitting emissions (sum of all hidden paths)
    """returns likelihood of emission string, given HMM"""

    S = len(states)
    n = len(emission)
    viterbi = np.zeros(shape = (S, n))

    # init first column of viterbi with Pr_emission & 1/States
    for state in range(S):
        viterbi[state][0] = 1/S * E[state][emission[0]]

    # Fill viterbi graph with sums over all incoming edges for ea node
    for i in range(1,n):
        for state in range(S):
            em = E[state][emission[i]]
            for prev in range(S):
                trans = T[prev][state]
                viterbi[state][i] += trans * em * viterbi[prev][i-1]
    return sum(viterbi[s][n-1] for s in range(S))

In [ ]:
with open('dataset_925973_4.txt') as f:
    lines = [line.strip() for line in f]
    emission, alphabet, states, T, E = parse_HMM(lines)
    T = np.exp(T)
    E = np.exp(E)

    print('string:', emission,'\nstates',states,'\n\nE_matrix\n',E, '\n\nT_matrix\n', T)


outcome_likelihood(emission, T, E, states)

10.11 Q4

In [ ]:
import pandas as pd


In [ ]:
def parameter_estimation(x, pi, states, alphabet):

    S = len(states)
    n = len(x)

    alphaperstate = {}
    for state in states:
        i = 0
        for ltr in pi:
            if(state == ltr and state in alphaperstate.keys()):
                alphaperstate[state].append(x[i])
            elif(state == ltr):
                alphaperstate[state] = [x[i]]
            i+=1



    emission = {}
    emission_data = {}

    used_alpha = []
    used_state = []

    for state in states:
        for alpha in alphabet:
            for key, value in alphaperstate.items():
                for i in range(len(value)):
                    if(value[i] == alpha and key == state and (state,alpha) in emission.keys()):
                        emission[state,alpha] += 1
                    elif(value[i] == alpha and key == state ):
                        emission[state,alpha] = 1
                        used_alpha.append(alpha)
                        used_state.append(state)

    if (len(used_state)!= len(states)*len(alphabet)):
        for state in states:
            if(state not in used_state):
                for alpha in alphabet:
                    emission[state,alpha] = 1
    '''
    if(len(used_alpha) != len(states)*len(alphabet)):
        for alpha in alphabet:
            if(alpha not in used_alpha):
                for state in states:
                    emission[state,alpha] = 1'''




    for state in states:
        for alpha in alphabet:
            if((state,alpha) in emission.keys()):
                if(state in alphaperstate.keys()):
                    emission[state,alpha] = emission[state,alpha] / len(alphaperstate[state])
                else:
                    emission[state,alpha] = emission[state,alpha] / len(alphabet)
                if(alpha in emission_data.keys()):
                    emission_data[alpha].append(emission[state,alpha])
                else:
                    emission_data[alpha] = [emission[state,alpha]]

    emission_df = pd.DataFrame(emission_data, index = states)

    transition = {}
    for state in states:
        for state2 in states:
            transition[state,state2] = 0
    for i in range(1, len(pi)):
        transition[pi[i-1], pi[i]] += 1

    for state in states:
        den = 0
        for key, value in transition.items():
            if(state == key[0]):
                den += value
        if(den == 0):
            den = 1

        for key, value in transition.items():
            if(state == key[0]):
                transition[key] = transition[key] / den

    transition_data = {}
    for state in states:
        for key, value in transition.items():
            if(state in key[1]):
                if(state in transition_data.keys()):
                    transition_data[state].append(value)
                else:
                    transition_data[state] = [value]
    transition_df = pd.DataFrame(transition_data, index = states)

    return(transition_df, emission_df)









In [ ]:
x = 'zxyyyxxyyzzzxxyzyyyyxzyzxxzxxyzzxzyzzyyyzyxyyyzzyyyxyyxxzzxxxzyzxyzyyyzzzyzxxyzzzzxzyyzyyzyyzxzyxyyz'
pi = 'BBBABBABBBCCBAAACBAACADDBDBCCADCDAADABCCCABAABCBBDCDCDDBACDCBBDDBCDADDCDABDBDAADCCCCCDCDDCDDCDDBBDBA'
states = ['A','B','C', 'D']
alphabet = ['x', 'y','z']

In [ ]:
parameter_estimation(x, pi, states, alphabet)

(          A         B         C         D
 A  0.300000  0.300000  0.150000  0.250000
 B  0.269231  0.307692  0.192308  0.230769
 C  0.120000  0.160000  0.320000  0.400000
 D  0.178571  0.250000  0.321429  0.250000,
           x         y         z
 A  0.095238  0.523810  0.380952
 B  0.384615  0.423077  0.192308
 C  0.200000  0.280000  0.520000
 D  0.250000  0.428571  0.321429)

In [ ]:
alphaperstate

In [ ]:
from pandas.io.stata import value_label_mismatch_doc
test = {('A', 'A'): 0,
 ('A', 'B'): 4,
 ('A', 'C'): 0,
 ('B', 'A'): 4,
 ('B', 'B'): 1,
 ('B', 'C'): 0,
 ('C', 'A'): 0,
 ('C', 'B'): 0,
 ('C', 'C'): 0}

for state in states:
    den = 0
    for key, value in test.items():
        if(state == key[0]):
            den += value
    if(den == 0):
        den = 1

    for key, value in test.items():
        if(state == key[0]):
            test[key] = test[key] / den

test

{('A', 'A'): 0.0,
 ('A', 'B'): 1.0,
 ('A', 'C'): 0.0,
 ('B', 'A'): 0.8,
 ('B', 'B'): 0.2,
 ('B', 'C'): 0.0,
 ('C', 'A'): 0.0,
 ('C', 'B'): 0.0,
 ('C', 'C'): 0.0}

10.12 Q5

In [ ]:
with open('dataset_925973_4.txt') as f:
    lines = [line.strip() for line in f]
    emission, alphabet, states, T, E = parse_HMM(lines)
    T = np.exp(T)
    E = np.exp(E)

    print('string:', emission,'\nstates',states,'\n\nE_matrix\n',E, '\n\nT_matrix\n', T)


den = outcome_likelihood(emission, T, E, states)

In [ ]:
class SoftDecoding:
    def __init__(self):
        x, transition, emission, alphabet, states = self.readFromFile()
        Pr = self.softDecode(x, transition, emission, alphabet, states)
        self.savePr(Pr, states)

    def readFromFile(self):
        f = open('dataset_925978_5.txt', 'r')
        data = f.read().strip().split()
        x = data[0]
        ind = [i for i in range(len(data)) if '--------' == data[i]]
        alphabet = data[ind[0]+1:ind[1]]
        states = data[ind[1]+1:ind[2]]
        transition = np.zeros((len(states), len(states)), dtype = float)
        emission = np.zeros((len(states), len(alphabet)), dtype = float)
        for i in range(len(states)):
            transition[i, :] = [float(d) for d in data[ind[2]+len(states)+2+i*(len(states)+1):ind[2]+len(states)+1+(i+1)*(len(states)+1)]]
            emission[i, :] = [float(d) for d in data[ind[3]+len(alphabet)+2+i*(len(alphabet)+1):ind[3]+len(alphabet)+1+(i+1)*(len(alphabet)+1)]]
        return x, transition, emission, alphabet, states

    def softDecode(self, x, transition, emission, alphabet, states):
        n = len(x)
        l = transition.shape[0]
        x2ind = {alphabet[i]:i for i in range(len(alphabet))}
        xList = [x2ind[x[i]] for i in range(len(x))]
        forward = [[0 for _ in range(l)] for __ in range(n)]
        backward = [[0 for _ in range(l)] for __ in range(n)]
        for k in range(l):
            forward[0][k] = emission[k, xList[0]]/l
        for i in range(1, n):
            for k in range(l):
                forward[i][k] = sum([forward[i-1][kpre]*transition[kpre, k]*emission[k, xList[i]] for kpre in range(l)])
        fsink = sum(forward[n-1])

        for k in range(l):
            backward[n-1][k] = 1
        for i in range(n-2, -1, -1):
            for k in range(l):
                backward[i][k] = sum([backward[i+1][kpre]*transition[k, kpre]*emission[kpre, xList[i+1]] for kpre in range(l)])

        Pr = np.zeros((n, l), dtype = float)
        for i in range(n):
            for k in range(l):
                Pr[i, k] = forward[i][k]*backward[i][k]/fsink

        return Pr

    def savePr(self, Pr, states):
        f = open('result.txt', 'w')
        print(' '.join(states))
        f.write('\t'.join(states)+'\n')
        for i in range(Pr.shape[0]):
            print(' '.join(list(map(str, Pr[i, :]))))
            f.write('\t'.join(list(map(str, Pr[i, :])))+'\n')
        f.close()

if __name__ == '__main__':
    SoftDecoding()

A B C
0.47905180894564475 0.30343336655366654 0.21751482450068846
0.19249753499405356 0.6208438469096077 0.1866586180963386
0.08981368924511879 0.35770384304666797 0.5524824677082133
0.1255008633330837 0.30195741867261333 0.5725417179943032
0.339361498923966 0.37797528855344054 0.2826632125225936
0.3538199974495944 0.38047121591073435 0.26570878663967146
0.3168653652592422 0.4232859560657042 0.2598486786750537
0.17781306392161222 0.6262654486881941 0.19592148739019372
0.15482179614436672 0.6279507061344333 0.2172274977212
0.23614306979266647 0.4682066953988062 0.29565023480852737


10.13 Q5

In [ ]:
class BaumWelch:
    def __init__(self):
        x, transition, emission, alphabet, states, iterNo = self.readFromFile()
        transition, emission = self.BaumWelchLearning(x, transition, emission, alphabet, states, iterNo)
        self.saveTransitionAndEmission(alphabet, states, transition, emission)

    def readFromFile(self):
        f = open('dataset_925979_5.txt', 'r')
        data = f.read().strip().split()
        iterNo = int(data[0])
        x = data[2]
        ind = [i for i in range(len(data)) if '--------' == data[i]]
        alphabet = data[ind[1]+1:ind[2]]
        states = data[ind[2]+1:ind[3]]
        transition = np.zeros((len(states), len(states)), dtype = float)
        emission = np.zeros((len(states), len(alphabet)), dtype = float)
        for i in range(len(states)):
            transition[i, :] = [float(d) for d in data[ind[3]+len(states)+2+i*(len(states)+1):ind[3]+len(states)+1+(i+1)*(len(states)+1)]]
            emission[i, :] = [float(d) for d in data[ind[4]+len(alphabet)+2+i*(len(alphabet)+1):ind[4]+len(alphabet)+1+(i+1)*(len(alphabet)+1)]]
        return x, transition, emission, alphabet, states, iterNo

    def softDecode(self, xList, transition, emission):
        n = len(xList)
        l = transition.shape[0]
        forward = [[0 for _ in range(l)] for __ in range(n)]
        backward = [[0 for _ in range(l)] for __ in range(n)]
        for k in range(l):
            forward[0][k] = emission[k, xList[0]]/l
        for i in range(1, n):
            for k in range(l):
                forward[i][k] = sum([forward[i-1][kpre]*transition[kpre, k]*emission[k, xList[i]] for kpre in range(l)])
        fsink = sum(forward[n-1])

        for k in range(l):
            backward[n-1][k] = 1
        for i in range(n-2, -1, -1):
            for k in range(l):
                backward[i][k] = sum([backward[i+1][kpre]*transition[k, kpre]*emission[kpre, xList[i+1]] for kpre in range(l)])

        Pr = np.zeros((l, n), dtype = float)
        for i in range(n):
            for k in range(l):
                Pr[k, i] = forward[i][k]*backward[i][k]/fsink

        Pr2 = np.zeros((l, l, n-1), dtype = float)
        for k1 in range(l):
            for k2 in range(l):
                for i in range(n-1):
                    Pr2[k1, k2, i] = forward[i][k1]*transition[k1, k2]*emission[k2, xList[i+1]]*\
                    backward[i+1][k2]/fsink

        return Pr, Pr2

    def estimateParameters(self, xList, Pr, Pr2, nAlphabet):
        n = len(xList)
        l = Pr2.shape[0]
        transition = np.zeros((l, l), dtype = float)
        emission = np.zeros((l, nAlphabet), dtype = float)
        for k1 in range(l):
            for k2 in range(l):
                transition[k1, k2] = sum(Pr2[k1, k2, :])
        for k in range(l):
            for i in range(n):
                emission[k, xList[i]] += Pr[k, i]

        for i in range(l):
            sum1 = sum(transition[i,:])
            if 0 == sum1:
                transition[i,:] += 1/l
            else:
                transition[i,:] /= sum1
            sum2 = sum(emission[i,:])
            if 0 == sum2:
                emission[i,:] += 1/nAlphabet
            else:
                emission[i,:] /= sum2
        return transition, emission

    def BaumWelchLearning(self, x, transition, emission, alphabet, states, iterNo):
        x2ind = {alphabet[i]:i for i in range(len(alphabet))}
        xList = [x2ind[x[i]] for i in range(len(x))]
        for _ in range(iterNo):
            Pr, Pr2 = self.softDecode(xList, transition, emission)
            transition, emission = self.estimateParameters(xList, Pr, Pr2, len(alphabet))
        return transition, emission

    def saveTransitionAndEmission(self, alphabet, states, fullTransition, emission):
        f = open('result.txt', 'w')
        print(' '.join([' '] + states))
        f.write('\t'+'\t'.join(states) + '\n')
        for i in range(fullTransition.shape[0]):
            print(' '.join([states[i]] + ['{:.3f}'.format(a) for a in fullTransition[i, :]]))
            f.write('\t'.join([states[i]] + ['{:.3f}'.format(a) for a in fullTransition[i, :]]) + '\n')
        print('--------')
        f.write('--------'+'\n')
        print(' '.join([' '] + alphabet))
        f.write('\t'+'\t'.join(alphabet)+'\n')
        for i in range(emission.shape[0]):
            print(' '.join([states[i]] + ['{:.3f}'.format(a) for a in emission[i, :]]))
            f.write('\t'.join([states[i]] + ['{:.3f}'.format(a) for a in emission[i, :]])+'\n')
        f.close()

if __name__ == '__main__':
    BaumWelch()